In [196]:
import numpy as np
from numpy import linalg as LA

In [197]:

def matrix_product_state_decomposition(tensor, bond_dim):
    # Get the shape of the input tensor
    shape = tensor.shape
    
    # first step 
    tensor = np.reshape(tensor, (shape[0], -1))
    U, S, V = np.linalg.svd(tensor, full_matrices=False)
    U = U[:, :bond_dim]
    S = np.diag(S[:bond_dim])
    V = V[:bond_dim, :]
    U = np.reshape(U, (shape[0], bond_dim))
    V = np.reshape(V, (bond_dim, -1))

    # Initialize left
    left_tensors = [U] # Start with identity matrix

    # Initialize list to hold core tensors
    core_tensors = [S]
    
    #next step

    tensor = V

    # Loop through each mode of the tensor
    for i in range(1,len(shape)-1):
        # Reshape the tensor to matrix form
        tensor = np.reshape(tensor, (bond_dim*shape[i],-1))

        # Perform singular value decomposition
        U, S, V = np.linalg.svd(tensor, full_matrices=False)
        
        # Truncate the singular values based on bond dimension
        U = U[:, :bond_dim]
        S = np.diag(S[:bond_dim])
        V = V[:bond_dim, :]

        # Reshape U and V to tensor form
        U = np.reshape(U, (bond_dim, shape[i], bond_dim))
        V = np.reshape(V, (bond_dim, -1))
        

        # Store the core tensor
        core_tensors.append(S)

        # Append left  tensors for the next iteration
        left_tensors.append(U)

        tensor = V

    V = np.reshape(V, (bond_dim, shape[-1]))
        
    return left_tensors, core_tensors, V


In [198]:

# Example usage
tensor = np.random.rand(2, 2, 2) # Example tensor
bond_dim = 2 # Bond dimension
left_tensors, core_tensors, right_tensor = matrix_product_state_decomposition(tensor, bond_dim)

# Print the shapes of left, core, and right tensors
print("Left tensors shapes:")
for t in left_tensors:
    print(t.shape)

print("\nCore tensors shapes:")
for t in core_tensors:
    print(t.shape)

print("\nRight tensor shape:")
print(right_tensor.shape)



Left tensors shapes:
(2, 2)
(2, 2, 2)

Core tensors shapes:
(2, 2)
(2, 2)

Right tensor shape:
(2, 2)


In [195]:
# Reconstruct the tensor
recon = left_tensors[0] 

for i in range(1,len(left_tensors)):
    recon = (recon.reshape(-1, bond_dim) @ core_tensors[i-1]) @ left_tensors[i].reshape(bond_dim,-1)
    print(recon.shape)
recon = (recon.reshape(-1,bond_dim) @ core_tensors[-1]) @ right_tensor

# Check the difference between the original tensor and the reconstructed tensor
difference = LA.norm(tensor - recon.reshape(tensor.shape)) 
print("Relative Frobenius norm of the difference between original tensor and reconstructed tensor:", difference)


(2, 4)
Relative Frobenius norm of the difference between original tensor and reconstructed tensor: 6.093992464111428e-16
